In [7]:
import sys
import os
import numpy as np
import igraph as ig

sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

from simulation import run_simulation, generate_network, _initialize_posts_and_weights

In [9]:
PARAMS = {
    "time_decay_rate": np.log(2)/2,  # halving the weight every 2 time steps. change /2 -> /k for halving every k time steps.
    "W_agent_success": 1,
    "W_personal_weights": 1,
    "W_post_success" : 1,
    "noise_level": 0.01
}
timesteps = 100
num_agents = 100
fill_history = 5


G = generate_network(N = num_agents, k = 4, p = 0.2, timesteps = timesteps, fill_history = fill_history)
POSTS, WEIGHTS, READ_MATRIX, LIKES = _initialize_posts_and_weights(G, debug = True)


G, POSTS, WEIGHTS, READ_MATRIX, LIKES = run_simulation(G, WEIGHTS, READ_MATRIX, LIKES, PARAMS, POSTS, fill_history, timesteps)
# this is the first timestep that we are simulating, so this is our "time 0", but we have filled some of the history to have meaningfull pasts.
# print the average number of likes for each agent:
print(np.mean(LIKES, axis=1))

[0.65714286 0.74285714 0.96190476 0.83809524 0.72380952 0.9047619
 1.16190476 0.6952381  1.3047619  1.21904762 0.38095238 0.85714286
 0.77142857 2.05714286 0.68571429 0.58095238 0.53333333 1.2952381
 1.58095238 0.75238095 1.06666667 0.76190476 0.73333333 0.74285714
 0.62857143 1.11428571 0.57142857 0.81904762 2.11428571 0.75238095
 0.60952381 1.15238095 1.00952381 0.94285714 0.79047619 0.66666667
 1.68571429 0.58095238 0.8        0.62857143 1.0952381  1.05714286
 1.02857143 0.86666667 0.87619048 0.99047619 0.95238095 0.79047619
 0.86666667 1.51428571 0.76190476 0.99047619 0.48571429 1.45714286
 0.6952381  0.98095238 0.94285714 0.8        0.51428571 0.96190476
 0.83809524 1.80952381 0.85714286 0.79047619 0.85714286 0.68571429
 0.51428571 0.72380952 1.7047619  1.77142857 0.56190476 0.6
 0.68571429 1.27619048 0.60952381 1.08571429 1.68571429 0.72380952
 0.97142857 0.48571429 0.65714286 1.13333333 0.60952381 1.5047619
 0.67619048 0.41904762 1.26666667 0.59047619 1.12380952 1.36190476
 1.23